In [1]:
from meteostat import Stations, Monthly
from datetime import datetime
from sqlalchemy import create_engine, Column, String, Float, Integer, Date, ForeignKey
from sqlalchemy.orm import sessionmaker, declarative_base, relationship
import pandas as pd
from tqdm.notebook import tqdm
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut

In [2]:
DATABASE_URL = "sqlite:///weather_data.db"
Base = declarative_base()
engine = create_engine(DATABASE_URL)
Session = sessionmaker(bind=engine)
session = Session()

In [3]:
class Station(Base):
    __tablename__ = 'stations'

    id = Column(String, primary_key=True)
    name = Column(String)
    country = Column(String)
    region = Column(String, nullable=True)
    city = Column(String, nullable=True)
    latitude = Column(Float)
    longitude = Column(Float)
    elevation = Column(Float)
    timezone = Column(String)
    daily_start = Column(Date)
    daily_end = Column(Date)
    monthly_start = Column(Date)
    monthly_end = Column(Date)

    weather_data = relationship("Weather", back_populates="station")

class Weather(Base):
    __tablename__ = 'weather'

    id = Column(Integer, primary_key=True, autoincrement=True)
    station_id = Column(String, ForeignKey('stations.id'))
    date = Column(Date)
    temperature_avg = Column(Float, nullable=True)
    temperature_min = Column(Float, nullable=True)
    temperature_max = Column(Float, nullable=True)
    precipitation = Column(Float, nullable=True)
    # snow_depth = Column(Float, nullable=True)
    # wind_dir = Column(Float, nullable=True)
    wind_speed = Column(Float, nullable=True)
    # wind_gust = Column(Float, nullable=True)
    pressure = Column(Float, nullable=True)
    sunshine = Column(Float, nullable=True)

    station = relationship("Station", back_populates="weather_data")

Base.metadata.create_all(engine)

In [4]:
stations = Stations().fetch()

In [5]:
stations

,name,country,region,wmo,icao,latitude,longitude,elevation,timezone,hourly_start,hourly_end,daily_start,daily_end,monthly_start,monthly_end
id,,,,,,,,,,,,,,,
00FAY,Holden Agdm,CA,AB,71227,CXHD,53.1900,-112.2500,688.0,America/Edmonton,2020-01-01,2024-12-07,2002-11-01,2024-03-13,2003-01-01,2022-01-01
00TG6,Athabasca 1,CA,AB,<NA>,<NA>,54.7200,-113.2900,515.0,America/Edmonton,NaT,NaT,2000-01-01,2022-07-12,2000-01-01,2010-01-01
01001,Jan Mayen,NO,<NA>,01001,ENJA,70.9333,-8.6667,10.0,Europe/Oslo,1931-01-01,2025-01-07,1921-12-31,2025-03-06,1922-01-01,2022-01-01
01002,Grahuken,NO,SJ,01002,<NA>,79.7833,14.4667,0.0,Europe/Oslo,1986-11-09,2025-01-07,2010-10-07,2020-08-17,NaT,NaT
01003,Hornsund,NO,<NA>,01003,<NA>,77.0000,15.5000,10.0,Europe/Oslo,1985-06-01,2025-01-07,2009-11-26,2020-08-31,2016-01-01,2017-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZXYDC,Bonavista,CA,NL,71178,<NA>,48.6700,-53.1100,26.0,America/St_Johns,2020-01-01,2022-12-14,2006-07-09,2022-12-11,2006-01-01,2022-01-01
ZYC17,Kingston Airport,CA,ON,<NA>,<NA>,44.2300,-76.6000,92.0,America/Toronto,2020-01-01,2022-12-15,2018-10-29,2022-12-12,2019-01-01,2022-01-01
ZYITU,Selfridge Air National Guard Base,US,MI,<NA>,KMTC,42.6046,-82.8353,177.0,America/Detroit,2022-04-23,2025-03-12,2022-04-24,2022-04-25,NaT,NaT


In [6]:
stations = stations.drop(columns=['wmo', 'icao', 'hourly_start', 'hourly_end'])
stations = stations.dropna()
stations

,name,country,region,latitude,longitude,elevation,timezone,daily_start,daily_end,monthly_start,monthly_end
id,,,,,,,,,,,
00FAY,Holden Agdm,CA,AB,53.1900,-112.2500,688.0,America/Edmonton,2002-11-01,2024-03-13,2003-01-01,2022-01-01
00TG6,Athabasca 1,CA,AB,54.7200,-113.2900,515.0,America/Edmonton,2000-01-01,2022-07-12,2000-01-01,2010-01-01
01004,New Alesund II,NO,SJ,78.9167,11.9333,8.0,Europe/Oslo,1968-12-31,1997-03-01,1969-01-01,1974-01-01
01007,New Alesund,NO,SJ,78.9167,11.9333,0.0,Europe/Oslo,1973-03-28,2025-03-05,1974-01-01,2022-01-01
01015,Hekkingen Lighthouse,NO,TR,69.6000,17.8333,14.0,Europe/Oslo,1979-10-31,2021-09-06,1979-01-01,2021-01-01
...,...,...,...,...,...,...,...,...,...,...,...
ZWWW0,Diwopu / Urumqi / Dihua,CN,XJ,43.9000,87.4667,654.0,Asia/Urumqi,1990-08-27,2022-04-25,2006-01-01,2022-01-01
ZX07Q,Okotoks,CA,AB,50.7300,-113.9600,1081.0,America/Edmonton,2000-01-01,2022-12-02,2000-01-01,2021-01-01
ZXYDC,Bonavista,CA,NL,48.6700,-53.1100,26.0,America/St_Johns,2006-07-09,2022-12-11,2006-01-01,2022-01-01


In [11]:
geolocator = Nominatim(user_agent='weather_app')

def get_nearest_city(lat, lon):
    try:
        location = geolocator.reverse((lat, lon), exactly_one=True, timeout=10)
        if location:
            address = location.raw.get('address', {})
            city_name = address.get('city') or address.get('town') or address.get('village') or None
            print(city_name)
            return city_name
    except GeocoderTimedOut:
        print('Request time out')
        return None
    except Exception as e:
        print(e)
        return None

# tqdm.pandas()
# stations['city'] = stations.progress_apply(
#     lambda row: get_nearest_city(row['latitude'], row['longitude']) if row['country'] == 'RU' else None, axis=1
# )

In [7]:
for _, station in stations.iterrows():
    station_entry = Station(
        id=station.name,
        name=station['name'],
        country=station['country'],
        region=station['region'],
        # city=station['city'],
        latitude=station['latitude'],
        longitude=station['longitude'],
        elevation=station['elevation'],
        timezone=station['timezone'],
        daily_start=station['daily_start'],
        daily_end=station['daily_end'],
        monthly_start=station['monthly_start'],
        monthly_end=station['monthly_end']
    )
    session.add(station_entry)
session.commit()

In [8]:
ru_stations = stations[stations.country == 'RU']
ru_stations

,name,country,region,latitude,longitude,elevation,timezone,daily_start,daily_end,monthly_start,monthly_end
id,,,,,,,,,,,
20046,Polargmo Im. E. T. Krenkelja,RU,ARK,80.6167,58.0500,20.0,Europe/Moscow,1957-12-02,2025-03-06,1958-01-01,2021-01-01
20049,Tikhaya Bay,RU,NEN,80.3667,52.9167,46.0,Europe/Moscow,1966-01-01,1995-04-30,1966-01-01,1995-01-01
20069,Ostrov Vize,RU,TAY,79.5000,76.9833,10.0,Asia/Omsk,1945-11-01,2025-03-06,1945-01-01,2021-01-01
20087,Ostrov Golomjannyj,RU,KY,79.5500,90.6167,8.0,Asia/Krasnoyarsk,1936-04-01,2025-03-06,1936-01-01,2021-01-01
20274,Ostrov Uedinenija,RU,TAY,77.5000,82.2000,23.0,Asia/Novosibirsk,1936-01-01,1996-11-21,1936-01-01,1996-01-01
...,...,...,...,...,...,...,...,...,...,...,...
UWKS0,Cheboksary / Sin’yaly,RU,CU,56.0833,47.3500,170.0,Europe/Moscow,2008-10-28,2022-04-25,2017-01-01,2022-01-01
UWLL0,Ulyanovsk / Baratayevka,RU,ULY,54.2667,48.2167,141.0,Europe/Volgograd,2005-01-15,2022-04-25,2006-01-01,2022-01-01
UWLW0,Ulyanovsk / Loshchina,RU,ULY,54.4000,48.8000,77.0,Europe/Volgograd,2004-07-15,2022-04-24,2013-01-01,2022-01-01


In [ ]:
i = 1
total = len(ru_stations)
print(f'Total {total} stations')
for _, ru_station in ru_stations.iterrows():
    start = ru_station['monthly_start'].to_pydatetime()
    end = ru_station['monthly_end'].to_pydatetime()
    data_m = Monthly(ru_station.name, start, end)
    data = data_m.fetch()
    
    for _, data_row in data.iterrows():
        station_entry = Weather(
            station_id = ru_station.name,
            date = data_row.name,
            temperature_avg = data_row['tavg'],
            temperature_min = data_row['tmin'],
            temperature_max = data_row['tmax'],
            precipitation = data_row['prcp'],
            wind_speed = data_row['wspd'],
            pressure = data_row['pres'],
            sunshine = data_row['tsun']
        )
        session.add(station_entry)
    session.commit()
    if i % 50 == 0: print(i)
    i += 1

In [12]:
ru_stations = session.query(Station).filter(Station.country == 'RU').all()

for station in tqdm(ru_stations):
    city = get_nearest_city(station.latitude, station.longitude)
    if city:
        station.city = city
session.commit()

  0%|          | 0/1181 [00:00<?, ?it/s]

None
None
None
None
None
None
None
None
None
None
None
Диксон
None
Хатанга
Сёяха
None
None
None
None
None
None
None
Саскылах
городское поселение Тикси
Жилинда
None
None
Чокурдах
None
None
None
Североморск
Териберка
None
None
Верхнетуломский
Мурманск
None
Ловозеро
Островной
None
Ковдор
None
None
None
Краснощелье
None
None
None
None
None
None
Умба
None
None
None
Нижняя Пёша
Калевала
None
None
None
None
Мезень
None
Юшкозеро
Ругозеро
None
None
None
Колежма
None
None
None
Холмогоры
Пинега
Лешуконское
Койнас
Реболы
None
Сегежа
Онега
None
None
Сура
Солнечный
Ыджыдъяг
Вяртсиля
Медвежьегорск
None
Кондопога
None
Шенкурск
Верхняя Тойма
Яренск
Сортавала
Пряжа
Петрозаводск
Пудож
Вытегра
Каргополь
Няндома
Хорошевская
Котлас
Выборг
Токсово
Олонец
Лодейное Поле
Свирица
Новая Ладога
Винницы
Белозерск
Вожега
Великий Устюг
None
сельское поселение Амдерма
None
None
Дудинка
None
Мыс Каменный
Нарьян-Мар
None
None
None
Воркута
None
Тазовский
Игарка
Окунев Нос
Петрунь
Салехард
None
None
None
None
None
None
Пе